In [71]:

import numpy as np
import pandas as pd


import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime

#

from scipy import stats
from scipy.stats import skew, boxcox_normmax, norm
from scipy.special import boxcox1p

#

from typing import Dict, List, Tuple, Sequence


import numpy as np 
import pandas as pd 
import warnings
warnings.filterwarnings('ignore')
from scipy import stats
from scipy.stats import skew, boxcox_normmax, norm
from scipy.special import boxcox1p

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [72]:
pd.options.display.max_columns = 250
pd.options.display.max_rows = 250

train_data = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
test_data = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")

train_data.head()

In [73]:
train_data.describe().round(2)

In [74]:
import numpy as np

train_data.size

In [75]:
train_data.shape

In [76]:
train_data.isna().sum()

In [77]:
train_data['YrSold'].unique()


In [78]:
#Eliminamos las variables sin sentido. En este caso #Id"
#train_data.drop("Id", axis=1, inplace=True)
#test_data.drop('Id', axis=1, inplace=True)

train_data.head()

In [79]:
#Hacemos un analisis de correlación

#OBSERVACION: tanto en el grafico de correlación como en algunas funciones de aca arriba, -
# solo aparecen las variables "numéricas" Y NO LAS CATEGÓRICAS. 

import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import matplotlib.gridspec as gridspec
from matplotlib.ticker import MaxNLocator

plt.style.use('fivethirtyeight')


sns.set(font_scale=1.1)
correlation_train = train_data.corr()
mask = np.triu(correlation_train.corr())
plt.figure(figsize=(20, 20))
sns.heatmap(correlation_train,
            annot=True,
            fmt='.1f',
            cmap='coolwarm',
            square=True,
            mask=mask,
            linewidths=1,
            cbar=False)

plt.show()

In [80]:
#Analizamos las "variables NUMERICAS"

#Una de las mejores maneras de ver cómo afectan los precios de venta son los "diagramas de dispersión". 
#También estamos trazando líneas de regresión polinómicas para ver la tendencia general.-
# de esta manera podemos entender los valores numéricos y su importancia en el precio de venta,
#también es muy útil para --> detectar valores atípicos <--.


def srt_reg(y, df):
    fig, axes = plt.subplots(12, 3, figsize=(25, 80))
    axes = axes.flatten()

    for i, j in zip(df.select_dtypes(include=['number']).columns, axes):

        sns.regplot(x=i,
                    y=y,
                    data=df,
                    ax=j,
                    order=3,
                    ci=None,
                    color='#e74c3c',
                    line_kws={'color': 'black'},
                    scatter_kws={'alpha':0.4})
        j.tick_params(labelrotation=45)
        j.yaxis.set_major_locator(MaxNLocator(nbins=10))

        plt.tight_layout()

srt_reg('SalePrice', train_data)

In [81]:
# Eliminar valores atípicos (después de detectarlos a simple vista).


train_data = train_data.drop(train_data[(train_data['OverallQual'] < 5) & (train_data['SalePrice'] > 200000)].index)

train_data = train_data.drop(train_data[(train_data['GrLivArea'] > 4000)& (train_data['SalePrice'] < 200000)].index)

train_data = train_data.drop(train_data[(train_data['GarageArea'] > 1200)& (train_data['SalePrice'] < 200000)].index)

train_data = train_data.drop(train_data[(train_data['TotalBsmtSF'] > 3000)& (train_data['SalePrice'] > 320000)].index)

train_data = train_data.drop(train_data[(train_data['1stFlrSF'] < 3000)& (train_data['SalePrice'] > 600000)].index)

train_data = train_data.drop(train_data[(train_data['1stFlrSF'] > 3000)& (train_data['SalePrice'] < 200000)].index)


In [82]:
#Analizamos ahoras las "variables CATEGORICAS" en relación al "precio de venta" de las casas.

def srt_box(y, df):
    fig, axes = plt.subplots(14, 3, figsize=(25, 80))
    axes = axes.flatten()

    for i, j in zip(df.select_dtypes(include=['object']).columns, axes):

        sortd = df.groupby([i])[y].median().sort_values(ascending=False)
        sns.boxplot(x=i,
                    y=y,
                    data=df,
                    palette='plasma',
                    order=sortd.index,
                    ax=j)
        j.tick_params(labelrotation=45)
        j.yaxis.set_major_locator(MaxNLocator(nbins=18))

        plt.tight_layout()

srt_box('SalePrice', train_data)

In [83]:
# Extraemos la variable objetivo del "dataset de entrenamiento"

y = train_data['SalePrice'].reset_index(drop=True)
#train_data = train_data.drop(['SalePrice'], axis=1)


# Unimos los datos, para no tener que hacer 2 veces cada operacion de transformación

dataset = pd.concat([train_data, test_data]).reset_index(drop=True)
print(dataset.shape)

In [84]:
#Ahora vamos a ver los datos faltantes

#Lo que podemos hacer es verlos en seco(como esta mas arriba del codigo) o verlos en graficos para asi visualizarlos mejor

def missing_percentage(df):
    total = df.isnull().sum().sort_values(ascending=False)[df.isnull().sum().sort_values(ascending=False) != 0]
    percent = (df.isnull().sum().sort_values(ascending=False) / len(df) *
                     100)[(df.isnull().sum().sort_values(ascending=False) / len(df) *
                     100) != 0]
    return pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])


missing = missing_percentage(dataset)

fig, ax = plt.subplots(figsize=(20, 5))
sns.barplot(x=missing.index, y='Percent', data=missing, palette='Reds_r')
plt.xticks(rotation=90)

display(missing.T.style.background_gradient(cmap='Reds', axis=1))

In [85]:
#Pasamos a corregir este problema de los datos NaN y null

# Lista de NaN, incluidas las columnas donde NaN significa ninguno.
none_cols = ['Alley', 'PoolQC', 'MiscFeature', 'Fence', 'FireplaceQu', 'GarageType',
             'GarageFinish', 'GarageQual', 'GarageCond', 'BsmtQual', 'BsmtCond',
             'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'MasVnrType']

# Lista de NaN, incluidas las columnas donde NaN significa 0.
zero_cols = ['BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath',
             'BsmtHalfBath', 'GarageYrBlt', 'GarageArea', 'GarageCars', 'MasVnrArea']

# Lista de NaN, incluidas las columnas donde NaN realmente falta, se reemplazará con el modo.
freq_cols = ['Electrical', 'Exterior1st', 'Exterior2nd', 'Functional', 'KitchenQual',
             'SaleType', 'Utilities']


# Llenando la lista de columnas arriba:

for col in zero_cols:
    dataset[col].replace(np.nan, 0, inplace=True)

for col in none_cols:
    dataset[col].replace(np.nan, 'None', inplace=True)

for col in freq_cols:
    dataset[col].replace(np.nan, dataset[col].mode()[0], inplace=True)
    
#La caracteristica de "MSZoning" y"Lot Frontage" son un poco complicada -
# se lo rellenamos con el tipo más común de cada categoría - 
# no es perfecto, pero al menos disminuimos un poco la aleatoriedad.

dataset['MSZoning'] = dataset.groupby('MSSubClass')['MSZoning'].apply(
    lambda x: x.fillna(x.mode()[0]))

# Filling LotFrontage according to Neighborhood
dataset['LotFrontage'] = dataset.groupby(
    ['Neighborhood'])['LotFrontage'].apply(lambda x: x.fillna(x.median()))




In [86]:
# Convertimos tipos de variables que son "números", pero que deberian ser tratados como "strings"

dataset['MSSubClass'] = dataset['MSSubClass'].astype(str)
dataset['YrSold'] = dataset['YrSold'].astype(str)
dataset['MoSold'] = dataset['MoSold'].astype(str)

In [87]:
#Los valores realmente raros y que parecen no agregar mucho en general,-
# (aparecen menos de 10 veces en nuestras observaciones) entran en el grupo "Otros". - 
# Es reducción de la dimensionalidad

others = ['Condition1', 'Condition2', 'RoofMatl', 'Exterior1st', 'Exterior2nd',
          'Heating', 'Electrical', 'Functional', 'SaleType']

for col in others:
    mask = dataset[col].isin(dataset[col].value_counts()[dataset[col].value_counts() < 10].index)
    dataset[col][mask] = 'Other'



In [88]:
#Convertimos variables CATEGORICAS en NUMERICAS

#Lo que se hace es agarrar cada Feature y a todos sus posibles valores -> asigarle un número. 
#Donde 1, 2 lo es mas, y asi sucesivamente..

neigh_map = {
    'MeadowV': 1,
    'IDOTRR': 1,
    'BrDale': 1,
    'BrkSide': 2,
    'OldTown': 2,
    'Edwards': 2,
    'Sawyer': 3,
    'Blueste': 3,
    'SWISU': 3,
    'NPkVill': 3,
    'NAmes': 3,
    'Mitchel': 4,
    'SawyerW': 5,
    'NWAmes': 5,
    'Gilbert': 5,
    'Blmngtn': 5,
    'CollgCr': 5,
    'ClearCr': 6,
    'Crawfor': 6,
    'Veenker': 7,
    'Somerst': 7,
    'Timber': 8,
    'StoneBr': 9,
    'NridgHt': 10,
    'NoRidge': 10
}

dataset['Neighborhood'] = dataset['Neighborhood'].map(neigh_map).astype('Int64')
ext_map = {'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5}
dataset['ExterQual'] = dataset['ExterQual'].map(ext_map).astype('Int64')
dataset['ExterCond'] = dataset['ExterCond'].map(ext_map).astype('Int64')
bsm_map = {'None': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5}
dataset['BsmtQual'] = dataset['BsmtQual'].map(bsm_map).astype('Int64')
dataset['BsmtCond'] = dataset['BsmtCond'].map(bsm_map).astype('Int64')
bsmf_map = {
    'None': 0,
    'Unf': 1,
    'LwQ': 2,
    'Rec': 3,
    'BLQ': 4,
    'ALQ': 5,
    'GLQ': 6
}

dataset['BsmtFinType1'] = dataset['BsmtFinType1'].map(bsmf_map).astype('Int64')
dataset['BsmtFinType2'] = dataset['BsmtFinType2'].map(bsmf_map).astype('Int64')
heat_map = {'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5}
dataset['HeatingQC'] = dataset['HeatingQC'].map(heat_map).astype('Int64')
dataset['KitchenQual'] = dataset['KitchenQual'].map(heat_map).astype('Int64')
dataset['FireplaceQu'] = dataset['FireplaceQu'].map(bsm_map).astype('Int64')
dataset['GarageCond'] = dataset['GarageCond'].map(bsm_map).astype('Int64')
dataset['GarageQual'] = dataset['GarageQual'].map(bsm_map).astype('Int64')

In [89]:
# Creando nuevas características basadas en observaciones previas
#EJ: se creó una nueva característica que agrupa todos los tiipos de Baños en uno solo.

dataset['TotalSF'] = (dataset['BsmtFinSF1'] + dataset['BsmtFinSF2'] +
                       dataset['1stFlrSF'] + dataset['2ndFlrSF'])
dataset['TotalBathrooms'] = (dataset['FullBath'] +
                              (0.5 * dataset['HalfBath']) +
                              dataset['BsmtFullBath'] +
                              (0.5 * dataset['BsmtHalfBath']))

dataset['TotalPorchSF'] = (dataset['OpenPorchSF'] + dataset['3SsnPorch'] +
                            dataset['EnclosedPorch'] +
                            dataset['ScreenPorch'] + dataset['WoodDeckSF'])

dataset['YearBlRm'] = (dataset['YearBuilt'] + dataset['YearRemodAdd'])

# Fusión de calidad y condiciones

dataset['TotalExtQual'] = (dataset['ExterQual'] + dataset['ExterCond'])
dataset['TotalBsmQual'] = (dataset['BsmtQual'] + dataset['BsmtCond'] +
                            dataset['BsmtFinType1'] +
                            dataset['BsmtFinType2'])
dataset['TotalGrgQual'] = (dataset['GarageQual'] + dataset['GarageCond'])
dataset['TotalQual'] = dataset['OverallQual'] + dataset['TotalExtQual'] + dataset['TotalBsmQual'] + dataset[
        'TotalGrgQual'] + dataset['KitchenQual'] + dataset['HeatingQC']

# Creación de nuevas funciones mediante el uso de nuevos indicadores de calidad

dataset['QualGr'] = dataset['TotalQual'] * dataset['GrLivArea']
dataset['QualBsm'] = dataset['TotalBsmQual'] * (dataset['BsmtFinSF1'] +
                                                  dataset['BsmtFinSF2'])
dataset['QualPorch'] = dataset['TotalExtQual'] * dataset['TotalPorchSF']
dataset['QualExt'] = dataset['TotalExtQual'] * dataset['MasVnrArea']
dataset['QualGrg'] = dataset['TotalGrgQual'] * dataset['GarageArea']
dataset['QlLivArea'] = (dataset['GrLivArea'] -
                         dataset['LowQualFinSF']) * (dataset['TotalQual'])
dataset['QualSFNg'] = dataset['QualGr'] * dataset['Neighborhood']


# Creando algunas características simples

dataset['HasPool'] = dataset['PoolArea'].apply(lambda x: 1 if x > 0 else 0)
dataset['Has2ndFloor'] = dataset['2ndFlrSF'].apply(lambda x: 1 if x > 0 else 0)
dataset['HasGarage'] = dataset['QualGrg'].apply(lambda x: 1 if x > 0 else 0)
dataset['HasBsmt'] = dataset['QualBsm'].apply(lambda x: 1 if x > 0 else 0)
dataset['HasFireplace'] = dataset['Fireplaces'].apply(lambda x: 1 if x > 0 else 0)
dataset['HasPorch'] = dataset['QualPorch'].apply(lambda x: 1 if x > 0 else 0)

In [90]:
##Transformando los datos
#Algunos de los valores continuos no se distribuyen de manera uniforme y no se ajustan a la distribución normal- 
# podemos solucionarlos mediante el uso de enfoques de transformación de pareja. Vamos a usar boxcox

possible_skewed = ['LotFrontage', 'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
                    'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea',
                    'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
                    'ScreenPorch', 'PoolArea', 'LowQualFinSF', 'MiscVal']

# Encontrar la asimetría de las características numéricas.

skew_features = np.abs(dataset[possible_skewed].apply(lambda x: skew(x)).sort_values(
    ascending=False))

# Filtrado de características sesgadas.

high_skew = skew_features[skew_features > 0.3]

# Toma de índices de alto sesgo.

skew_index = high_skew.index

# Aplicar la transformación boxcox para corregir la asimetría.

for i in skew_index:
    dataset[i] = boxcox1p(dataset[i], boxcox_normmax(dataset[i] + 1))


In [91]:
#Codificación de etiquetas de las variables categóricas

dataset = pd.get_dummies(data=dataset)


In [92]:
#Chequeamos, antes de empezar con el modelado

dataset.drop(columns='SalePrice', inplace=True)
print(f'Number of missing values: {dataset.isna().sum().sum()}')

In [93]:
#----Entrenamiento----

# Separamos el dataset
train = dataset.iloc[:len(y), :]
test = dataset.iloc[len(train):, :]


print(len(test))
print(len(train))

In [94]:
X = train
X_test = test
y = np.log1p(y)

In [95]:
# Cargando paquetes necesarios para el modelado:

from sklearn.model_selection import cross_val_score, KFold, cross_validate
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV, TweedieRegressor
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import GradientBoostingRegressor, HistGradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from mlxtend.regressor import StackingCVRegressor

In [96]:
# Configuración de kfold para uso futuro.
kf = KFold(10, random_state=42, shuffle=True)

In [97]:
#Modelado


alphas_alt = [30.5, 20.6, 20.7, 20.8, 20.9, 20, 20.1, 20.2, 20.3, 20.4, 20.5]
alphas2 = [0.01]
e_alphas = [0.01]
e_l1ratio = [0.8, 0.85, 0.9, 0.95, 0.99, 1]

# ridge_cv:
ridge = make_pipeline(RobustScaler(), RidgeCV(
    alphas=alphas_alt,
    cv=kf,))

# lasso_cv:
lasso = make_pipeline(
    RobustScaler(),
    LassoCV(max_iter=1e3, alphas=alphas2, random_state=42, cv=kf))

# elasticnet_cv:
elasticnet = make_pipeline(
    RobustScaler(),
    ElasticNetCV(max_iter=1e3,
                 alphas=e_alphas,
                 cv=kf,
                 random_state=42,
                 l1_ratio=e_l1ratio))

# svr:
svr = make_pipeline(RobustScaler(),
                    SVR(C=21, epsilon=0.0099, gamma=0.00017, tol=0.000121))

# gradientboosting:
gbr = GradientBoostingRegressor(n_estimators=2900,
                                learning_rate=0.0161,
                                max_depth=4,
                                max_features='sqrt',
                                min_samples_leaf=17,
                                loss='huber',
                                random_state=42)

# lightgbm:
lightgbm = LGBMRegressor(objective='regression',
                         n_estimators=3500,
                         num_leaves=5,
                         learning_rate=0.00721,
                         max_bin=163,
                         bagging_fraction=0.35711,
                         n_jobs=-1,
                         bagging_seed=42,
                         feature_fraction_seed=42,
                         bagging_freq=7,
                         feature_fraction=0.1294,
                         min_data_in_leaf=8)

# xgboost:
xgboost = XGBRegressor(
    learning_rate =0.0139,
    n_estimators =4500,
    max_depth =4,
    min_child_weight =0,
    subsample =0.7968,
    colsample_bytree =0.4064,
    nthread =-1,
    scale_pos_weight =2,
    seed=42,
    enable_categorical=True)


# histgradientboost:
hgrd= HistGradientBoostingRegressor(loss= 'least_squares',
    max_depth = 2,
    min_samples_leaf = 40,
    max_leaf_nodes = 29,
    learning_rate = 0.15,
    max_iter = 500,
    random_state=42)

#tweedie regresson:
tweed = make_pipeline(RobustScaler(),TweedieRegressor(alpha=0.005))


# stacking regressor:
stack_gen = StackingCVRegressor(regressors=(ridge, lasso, elasticnet, gbr,
                                            xgboost, lightgbm,hgrd, tweed),
                                            meta_regressor=xgboost,
                                            use_features_in_secondary=True)

In [98]:
# Validación cruzada

def model_check(X, y, estimators, cv):
    
    ''' A function for testing multiple estimators.'''
    
    model_table = pd.DataFrame()

    row_index = 0
    for est, label in zip(estimators, labels):

        MLA_name = label
        model_table.loc[row_index, 'Model Name'] = MLA_name

        cv_results = cross_validate(est,
                                    X,
                                    y,
                                    cv=cv,
                                    scoring='neg_root_mean_squared_error',
                                    return_train_score=True,
                                    n_jobs=-1)

        model_table.loc[row_index, 'Train RMSE'] = -cv_results[
            'train_score'].mean()
        model_table.loc[row_index, 'Test RMSE'] = -cv_results[
            'test_score'].mean()
        model_table.loc[row_index, 'Test Std'] = cv_results['test_score'].std()
        model_table.loc[row_index, 'Time'] = cv_results['fit_time'].mean()

        row_index += 1
        print("--------")

    model_table.sort_values(by=['Test RMSE'],
                            ascending=True,
                            inplace=True)

    return model_table

In [99]:
# Setting list of estimators and labels for them.

estimators = [ridge, lasso, elasticnet, gbr, xgboost, lightgbm, svr, hgrd, tweed]
labels = [
    'Ridge', 'Lasso', 'Elasticnet', 'GradientBoostingRegressor',
    'XGBRegressor', 'LGBMRegressor', 'SVR', 'HistGradientBoostingRegressor','TweedieRegressor'
]

In [ ]:
# Executing cross validation.

raw_models = model_check(X, y, estimators, kf)
display(raw_models.style.background_gradient(cmap='summer_r'))

In [ ]:
# Fitting the models on train data:

#APILAR: El apilamiento es una técnica de aprendizaje de conjunto para combinar múltiples modelos de regresión 
#a través de un metarregresor

#Apilar y mezclar
#Aquí ajustamos cada estimador que tenemos en los datos del tren y luego 
#los combinamos asignando pesos a cada modelo y sumamos los resultados.
#Los pesos son bastante subjetivos y estoy bastante seguro de que puedes encontrar algo que 
#funcione mejor que esto si juegas con él...

print('=' * 20, 'START Fitting', '=' * 20)
print('=' * 55)

#print(datetime.now(), 'StackingCVRegressor')
#stack_gen_model = stack_gen.fit(X.values, y.values)

print(datetime.now(), 'Elasticnet')
elastic_model_full_data = elasticnet.fit(X, y)

print(datetime.now(), 'Lasso')
lasso_model_full_data = lasso.fit(X, y)

print(datetime.now(), 'Ridge')
ridge_model_full_data = ridge.fit(X, y)

print(datetime.now(), 'SVR')
svr_model_full_data = svr.fit(X, y)

print(datetime.now(), 'GradientBoosting')
gbr_model_full_data = gbr.fit(X, y)

#print(datetime.now(), 'XGboost')
#xgb_model_full_data = xgboost.fit(X, y)

#print(datetime.now(), 'Lightgbm')
#lgb_model_full_data = lightgbm.fit(X, y)

print(datetime.now(), 'Hist')
hist_full_data = hgrd.fit(X, y)

print(datetime.now(), 'Tweed')
tweed_full_data = tweed.fit(X, y)

print('=' * 20, 'FINISHED Fitting', '=' * 20)
print('=' * 58)

In [ ]:
# Obtengo las Predicciones y Evaluo la precisión de cada modelo (en 3 formas diferentes)

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

#ACLARACION: en este caso estan mal tomados los datos de entrenamiento aca abajo. Hay un error
#No lo pude corregir por diferencia entre una pequeña longitud entre X y X_test. Dejando eso de lado, el resto esta bien


#y1 = elastic_model_full_data.predict(X_test)
print("\nelastic_model_full_data")
y1 = elastic_model_full_data.predict(X)
print("MAE",mean_absolute_error(y1, y))
print("MSE",mean_squared_error(y1, y))
print("RMSE",np.sqrt(mean_squared_error(y1, y)))

print("\nlasso_model_full_data")
y2 = lasso_model_full_data.predict(X)
print("MAE",mean_absolute_error(y2, y))
print("MSE",mean_squared_error(y2, y))
print("RMSE",np.sqrt(mean_squared_error(y2, y)))

print("\nridge_model_full_data")
y3 = ridge_model_full_data.predict(X)
print("MAE",mean_absolute_error(y3, y))
print("MSE",mean_squared_error(y3, y))
print("RMSE",np.sqrt(mean_squared_error(y3, y)))

print("\nsvr_model_full_data")
y4 = svr_model_full_data.predict(X)
print("MAE",mean_absolute_error(y4, y))
print("MSE",mean_squared_error(y4, y))
print("RMSE",np.sqrt(mean_squared_error(y4, y)))

print("\ngbr_model_full_data")
y5 = gbr_model_full_data.predict(X)
print("MAE",mean_absolute_error(y5, y))
print("MSE",mean_squared_error(y5, y))
print("RMSE",np.sqrt(mean_squared_error(y5, y)))

print("\nhist_full_data")
y6 = hist_full_data.predict(X)
print("MAE",mean_absolute_error(y6, y))
print("MSE",mean_squared_error(y6, y))
print("RMSE",np.sqrt(mean_squared_error(y6, y)))

print("\ntweed_full_data")
y7 = tweed_full_data.predict(X)
print("MAE",mean_absolute_error(y7, y))
print("MSE",mean_squared_error(y7, y))
print("RMSE",np.sqrt(mean_squared_error(y7, y)))

In [ ]:
# Combinar modelos (dandole una ponderación a cada uno)

def blend_models_predict(X):
    return ((0.05 * elastic_model_full_data.predict(X)) +
            (0.05 * lasso_model_full_data.predict(X)) +
            (0.25 * ridge_model_full_data.predict(X)) +
            (0.25 * svr_model_full_data.predict(X)) +
            (0.05 * gbr_model_full_data.predict(X)) +
            (0.1 * hist_full_data.predict(X)) +
            (0.25 * tweed_full_data.predict(X)))

In [ ]:
#Nuestros modelos están ajustados, apilados y combinados para que 
#podamos predecir y enviar nuestros resultados.

submission = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

# Predicciones de precios de venta a escala de registro de inversión y suelo para ver los precios reales nuevamente.

submission['SalePrice'] = np.floor(np.expm1(blend_models_predict(X_test)))

# Creando dataframe de envío.

submission = submission[['Id', 'SalePrice']]

In [ ]:
# Guardando como un archivo csv:

submission.to_csv('mysubmission.csv', index=False)

print('Saving submission.',datetime.now(),)
submission.head()